# eICU-CRD tutorial

This notebook shows examples of how to use the cycquery tool on [eicu-CRD](https://eicu-crd.mit.edu/).

Each query is limit to 100 rows (for quick results).

* First, setup the eICU database according to the instructions in [eicu-code](https://github.com/MIT-LCP/eicu-code/tree/master/build-db/postgres).
* The database is assumed to be hosted using postgres. Update the config parameters such as username and password, passed to `EICUQuerier` accordingly.

## Imports and instantiate `EICUQuerier`

In [ ]:
"""eICU-CRD tutorial."""

import cycquery.ops as qo
from cycquery import EICUQuerier


querier = EICUQuerier(
    dbms="postgresql",
    port=5432,
    host="localhost",
    database="eicu",
    user="postgres",
    password="pwd",
)
# List all tables.
querier.list_tables("eicu_crd")

## Example 1. Get all female patients discharged in 2014 (limit to 100 rows).

In [ ]:
ops = qo.Sequential(
    qo.ConditionEquals("hospitaldischargeyear", 2014),
    qo.ConditionEquals("gender", "Female"),
)
patients = querier.eicu_crd.patient()
patients = patients.ops(ops).run(limit=100)
print(f"{len(patients)} rows extracted!")

## Example 2. Get all patient encounters with diagnoses (`schizophrenia` in `diagnosisstring`), discharged in the year 2015.

In [ ]:
patients = querier.eicu_crd.patient()
diagnoses = querier.eicu_crd.diagnosis()
diagnoses = diagnoses.ops(qo.ConditionSubstring("diagnosisstring", "schizophrenia"))
patient_diagnoses = patients.join(
    join_table=diagnoses,
    on="patientunitstayid",
)
patient_diagnoses = patient_diagnoses.run(limit=100)
print(f"{len(patient_diagnoses)} rows extracted!")

## Example 3. Get potassium lab tests for patients discharged in the year 2014, for all teaching hospitals.

In [ ]:
hospitals = querier.eicu_crd.hospital()
hospitals = hospitals.ops(qo.ConditionEquals("teachingstatus", True))
patients = querier.eicu_crd.patient()
patients = patients.ops(qo.ConditionEquals("hospitaldischargeyear", 2015))
patients = patients.join(
    join_table=hospitals,
    on="hospitalid",
)
labs = querier.eicu_crd.lab()
labs = labs.ops(qo.ConditionEquals("labname", "potassium"))
patient_labs = patients.join(
    join_table=labs,
    on="patientunitstayid",
).run(limit=100)
print(f"{len(patient_labs)} rows extracted!")

## Example 4. Get glucose medications (substring search) for female patients discharged in 2014.

In [ ]:
ops = qo.Sequential(
    qo.ConditionEquals("hospitaldischargeyear", 2014),
    qo.ConditionEquals("gender", "Female"),
)
patients = querier.eicu_crd.patient()
patients = patients.ops(ops)
medications = querier.eicu_crd.medication()
medications = medications.ops(qo.ConditionSubstring("drugname", "glucose"))
patient_medications = patients.join(
    join_table=medications,
    on="patientunitstayid",
).run(limit=100)
print(f"{len(patient_medications)} rows extracted!")